In [1]:
import pandas as pd
cell_matrix = pd.read_parquet('/home/adm808/NormalizedCellMatrixSyn18485175.parquet')


In [2]:
cell_matrix.head()

,AAACGGGAGATCCCGC.1,AAATGCCTCCAATGGT.1,AACCATGTCAGTGCAT.1,AACCATGTCTGTACGA.1,AACCGCGTCCGCATAA.1,AACGTTGGTTCAGGCC.1,AACTGGTGTACCGGCT.1,AAGACCTAGTTAACGA.1,AAGGAGCAGCAATCTC.1,AAGGAGCTCTGCTGCT.1,...,GCCTCTACACTTAACG.44,TCATTTGTCCGTAGGC.44,GATGAAACAGCGTAAG.45,GTGCGGTCAATCGGTT.46,ACTTGTTAGGAATTAC.48,AGGCCGTAGAGCAATT.48,AGTGAGGGTGCAACGA.48,CACACTCTCTCTGAGA.48,TAGTGGTAGAATGTTG.48,TGCCCATAGTAGGTGC.48
FO538757.2,0.0,6.919779,11.637401,0.000000,7.831354,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,9.763305,0.0,0.0,0.0,0.0,0.0
SAMD11,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
NOC2L,0.0,0.000000,0.000000,6.974242,0.000000,0.0,0.0,0.0,8.842948,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
KLHL17,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,9.377234,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
PLEKHN1,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [3]:
train_metadata = pd.read_parquet('/home/adm808/CellMetadataSyn18485175.parquet')

In [4]:
print(train_metadata.shape)

(70634, 8)


In [17]:
train_metadata.head()
train_metadata.shape

(70634, 8)

In [6]:
cell_type = 'Mic'

In [7]:
import numpy as np
def select_missing_genes(filtered_matrix):
    mean_threshold = 1
    missingness_threshold = 95
    mean_gene_expression = np.mean(filtered_matrix, axis=1)
    missingness = (filtered_matrix == 0).sum(axis=1) / filtered_matrix.shape[1] * 100
    null_expression = (missingness > missingness_threshold) & (mean_gene_expression < mean_threshold)
    genes_to_drop = filtered_matrix.index[null_expression].tolist()
    return genes_to_drop



In [13]:
def drop_missing_genes(matrix, cell_metadata, cell_type):
    cell_type_specific_metadata = cell_metadata[cell_metadata['broad.cell.type'] == cell_type]
    cell_names = cell_type_specific_metadata['TAG']
    matrix_filtered = matrix[cell_names]
    print(f'Number of genes and cells that are: {cell_type}{matrix_filtered.shape}')
    genes_to_drop = select_missing_genes(matrix_filtered)
    df_filtered = matrix_filtered.drop(genes_to_drop, axis=0)
    return df_filtered, cell_type_specific_metadata

train_matrix_filtered = drop_missing_genes(cell_matrix, train_metadata, cell_type)

Number of genes and cells that are: Mic(17926, 1920)


In [21]:
def filter_metadata(cell_metadata, cell_type):
    cell_specific_metadata = cell_metadata[cell_metadata['broad.cell.type'] == cell_type]
    return cell_specific_metadata
train_metadata_filtered = filter_metadata(train_metadata, cell_type)
#test_metadata_filtered = filter_metadata(test_metadata, cell_type)

In [22]:
print(train_metadata_filtered.shape)

(1920, 8)
